In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn
import os

# Data Pipeline

In [2]:
shakespeare_url = "https://homl.info/shakespeare"
filepath = tf.keras.utils.get_file('shakespeare.txt', shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [3]:
print(shakespeare_text[:80])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


## Encoding

In [4]:
text_vec_layer = tf.keras.layers.TextVectorization(split = 'character',
                                                   standardize = 'lower')
text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]

In [5]:
encoded

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([21,  7, 10, ..., 22, 28, 12], dtype=int64)>

In [6]:
encoded -= 2

In [7]:
n_tokens = text_vec_layer.vocabulary_size()
dataset_size = len(encoded)

In [8]:
dataset_size

1115394

In [9]:
from Utility import to_dataset
length = 100 
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1000000], length = length, shuffle=True, seed = 42)
valid_set = to_dataset(encoded[1000000: 1060000], length = length)
test_set = to_dataset(encoded[1060000:], length = length)

## Model

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = n_tokens, output_dim = 16),
    tf.keras.layers.GRU(128, return_sequences= True),
    tf.keras.layers.Dense(n_tokens, activation = 'softmax')
])

model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer = tf.keras.optimizers.Nadam(),
              metrics = ['accuracy'])

In [11]:
if os.path.exists("shakspeare_model.keras"):
    tf.keras.models.load_model("shakspeare_model.keras")
else:

    history = model.fit(train_set , validation_data= valid_set, epochs = 10,
                        callbacks = [tf.keras.callbacks.ModelCheckpoint('shakspeare_model.keras', monitor = 'val_accuracy', save_best_only= True)])

In [12]:
def input_vector(x):
    x = tf.constant(x)
    return x

In [13]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X -2),
    model
])

In [18]:
input_transformed = tf.constant(["To be or not to b"])
y_proba = shakespeare_model.predict(input_transformed)[0,-1]
y_pred = tf.argmax(y_proba)
text_vec_layer.get_vocabulary()[y_pred+2]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step


'-'

In [22]:
def next_char(text, temperature):
    y_proba = shakespeare_model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    vocab = text_vec_layer.get_vocabulary()
    # Ensure char_id + 2 does not exceed the bounds of the vocabulary list
    safe_char_id = min(char_id + 2, len(vocab) - 1)
    return vocab[safe_char_id]

def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [16]:
#def next_char(text, temperature):
#    y_proba = shakespeare_model.predict([text])[0,-1:]
#    rescaled_logits = tf.math.log(y_proba) / temperature
#    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0,0]
#    return text_vec_layer.get_vocabulary()[char_id + 2]

#def extend_text(text, n_chars = 50 , temperature  = 1):
#    for _ in range(n_chars):
#        text += next_char(text, temperature)
#    return text    

In [27]:
tf.random.set_seed(42)
print(extend_text(tf.constant(['To be or not to be']), temperature=0.01))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━